# Möbius Plateau Auto‑Explorer Ω+ (exhaustive)
Grandma‑friendly: paste your scan rows into **Cell 1** and press **Runtime ▸ Run all**.

### What it does
1. Reads your fine‑scan table *(bits, k, φ, λ, rel_err)*.
2. Tags rows into plateaux at three granularities *(ε = 1e‑4, 5e‑6, 1e‑5).*.
3. Applies a comprehensive bag‑of‑ops on each plateau:
   * `×9, ÷9, ×3, ÷3, ×6, ÷6, ×α, ÷α, ×α½, ÷α½, ×α², ÷α²`
   * Same for φ (golden ratio), δ (Feigenbaum δ ≈ 4.669), α_F (Feigenbaum α ≈ 2.5029)
   * Complements `1 – c` for every constant.
4. Prints the first 60 rows for *every* `(ε, op)` so you can eyeball structure.
5. Saves **two Excel files** in `/content`:
   * **plateaus_raw.xlsx** – untouched rows.
   * **plateaus_ops.xlsx** – one sheet per `(ε, op)`.

_φ = 0.00394344 ± 5×10⁻⁷ ; α⁻¹ = 137.035999084 ; δ ≈ 4.669201609 ; α_F ≈ 2.502907875_

**No widgets, no parameters — just run.**

In [9]:

import pandas as pd, numpy as np

# ▶▶ Paste / update your raw rows here ◀◀
raw_rows = [
    #  bits   k     φ-micro      rel_err
    (64,  3.6, 0.003942, 0.4117525875906678269),
    (64,  3.8, 0.003942, 0.4117525875906678269),
    (64,  4.0, 0.003942, 0.4117525875906678269),
    (64,  4.2, 0.003942, 0.4117525875906678269),
    (128, 3.6, 0.003942, 0.4117525875906678271),
    (128, 3.8, 0.003942, 0.4117525875906678271),
    (128, 4.0, 0.003942, 0.4117525875906678271),
    (128, 4.2, 0.003942, 0.4117525875906678271),
]

df = pd.DataFrame(raw_rows, columns=['bits','k','phi','rel_err'])
print("Preview →")
display(df.head())


Preview →


,bits,k,phi,rel_err
0,64,3.6,0.003942,0.411753
1,64,3.8,0.003942,0.411753
2,64,4.0,0.003942,0.411753
3,64,4.2,0.003942,0.411753
4,128,3.6,0.003942,0.411753


In [10]:
# === built‑in constants & op lambdas ===
import math, itertools, numpy as np

phi = (1+5**0.5)/2
phi_inv = 1/phi
alpha_inv = 137.035999084   # fine‑structure 1/α
alpha    = 1/alpha_inv
delta    = 4.669201609      # Feigenbaum δ
alpha_F  = 2.502907875      # Feigenbaum α

# helper inverses / complements
def compl(c): return 1-c

consts = {
    '9':9.0,'3':3.0,'6':6.0, # Ensure these are floats
    'phi':phi,'phi½':phi**0.5,'phi²':phi**2,'phi⁻¹':phi_inv,'φ':phi, # Use numeric value for 'φ'
    'alpha':alpha,'alpha½':alpha**0.5,'alpha²':alpha**2,'α':alpha, # Use numeric value for 'α'
    'alpha_inv':alpha_inv,'α⁻¹':alpha_inv, # Use numeric value for 'α⁻¹'
    'delta':delta,'delta½':delta**0.5,'delta²':delta**2,'δ':delta, # Use numeric value for 'δ'
    'alpha_F':alpha_F,'alpha_F½':alpha_F**0.5,'alpha_F²':alpha_F**2,
}

# compose lambdas
OPS = {}
OPS['×c'] = lambda x, c=None: x*c if c is not None else x # Fixed: Added default argument
OPS['÷c'] = lambda x, c=None: x/c if c is not None and c != 0 else x # Fixed: Added default argument and div by zero check
for name,c in consts.items():
    # multiply & divide by constants; c may be str for numbers too
    OPS['×'+name] = lambda x, c=c: x*float(c)
    OPS['÷'+name] = lambda x, c=c: x/float(c)
    OPS['+'+name] = lambda x, c=c: x+float(c)
    OPS['-'+name] = lambda x, c=c: x-float(c)
    OPS['c_'+name] = lambda x, c=c: compl(float(c))   # constant standalone for sanity

print(f"Loaded {len(OPS)} ops")

Loaded 107 ops


In [11]:

# === plateau tagging & exhaustive run ===
from itertools import product
import pandas as pd, numpy as np, math, os

def tag_plateaus(frame: pd.DataFrame, eps: float):
    base = frame['rel_err'].min()
    out  = frame.copy()
    out['plateau'] = ( (out['rel_err']-base)/eps ).astype(int)
    return out

views = []
for eps in [1e-4,5e-6,1e-5]:
    tagged = tag_plateaus(df, eps)
    for op_name, op_fun in OPS.items():
        tmp = tagged.copy()
        tmp['ε'] = eps
        tmp['op'] = op_name
        tmp['rel_err'] = op_fun(tmp['rel_err'])
        views.append(tmp)
        print(f"ε={eps:g}, op={op_name} → first row rel_err={tmp.rel_err.iloc[0]:.6g}")

out = pd.concat(views, ignore_index=True)

# save
raw_file   = 'plateaus_raw.xlsx'
ops_file   = 'plateaus_ops.xlsx'
df.to_excel(raw_file, index=False)
out.to_excel(ops_file, index=False)
print(f'Wrote {raw_file} and {ops_file}')


ε=0.0001, op=×c → first row rel_err=0.411753
ε=0.0001, op=÷c → first row rel_err=0.411753
ε=0.0001, op=×9 → first row rel_err=3.70577
ε=0.0001, op=÷9 → first row rel_err=0.0457503
ε=0.0001, op=+9 → first row rel_err=9.41175
ε=0.0001, op=-9 → first row rel_err=-8.58825
ε=0.0001, op=c_9 → first row rel_err=-8
ε=0.0001, op=×3 → first row rel_err=1.23526
ε=0.0001, op=÷3 → first row rel_err=0.137251
ε=0.0001, op=+3 → first row rel_err=3.41175
ε=0.0001, op=-3 → first row rel_err=-2.58825
ε=0.0001, op=c_3 → first row rel_err=-2
ε=0.0001, op=×6 → first row rel_err=2.47052
ε=0.0001, op=÷6 → first row rel_err=0.0686254
ε=0.0001, op=+6 → first row rel_err=6.41175
ε=0.0001, op=-6 → first row rel_err=-5.58825
ε=0.0001, op=c_6 → first row rel_err=-5
ε=0.0001, op=×phi → first row rel_err=0.66623
ε=0.0001, op=÷phi → first row rel_err=0.254477
ε=0.0001, op=+phi → first row rel_err=2.02979
ε=0.0001, op=-phi → first row rel_err=-1.20628
ε=0.0001, op=c_phi → first row rel_err=-0.618034
ε=0.0001, op=×phi½ 